# Introduction

- A spatial model is definde in the following way:
$$
y_{it} = \beta X_{it} + \rho \sum_{j=1}^{N} w_{it}
$$

In [ ]:
import os
os.chdir("..")

In [ ]:
import numpy as np
import pymc as pm
import arviz as az
import bambi as bmb
import statsmodels.api as sm
from pygam import LinearGAM, s, te
import matplotlib.pyplot as plt
from src.data.data_reg import SpatialReg
az.style.use("arviz-darkgrid")


num = 10
rho = .8
sr = SpatialReg()


In [ ]:
gdf = sr.spatial_panel(n=num,time=50,rho=rho)
gdf

In [ ]:
gdf[gdf["time"]==0].plot("X1")

In [ ]:
gdf[gdf["time"]==1].plot("y_d")

In [ ]:
xb = gdf[["X1", "w_d"]].values.reshape(-1,2)
y_d = gdf["y_d"].values.reshape(-1,1)
X = sm.add_constant(xb)
results = sm.OLS(y_d, X).fit()
print(results.summary())

In [ ]:
df = gdf.drop("geometry", axis=1)
priors = {
    "w_d": bmb.Prior("Normal", mu=0, sigma=2),
}
model = bmb.Model(
    "y_d ~ 1 + X1 + w_d",
    priors=priors,
    data=df, 
    dropna=True
)
results = model.fit()

In [ ]:
az.plot_trace(results)
az.summary(results)

In [ ]:
gdf["centroid"] = gdf.centroid
gdf["lat"] = gdf["centroid"].x
gdf["lon"] = gdf["centroid"].y
df = gdf.drop("geometry", axis=1)
df

In [ ]:
# Simulate data
np.random.seed(42)
n = 500

# Covariates
longitude = np.random.uniform(-10, 10, n)
latitude = np.random.uniform(-10, 10, n)
X1 = np.random.normal(0, 1, n)

# True spatial function (nonlinear)
def spatial_effect(lon, lat):
    return np.sin(lon / 2) + np.cos(lat / 3)

# Response variable
y = 3 + 2 * X1 + spatial_effect(longitude, latitude) + np.random.normal(0, 1, n)

# Create data matrix for pyGAM
X = np.column_stack([X1, longitude, latitude])

# Fit GAM with tensor product smooth over (longitude, latitude)
gam = LinearGAM(te(1, 2) + s(0)).fit(X, y)


In [ ]:
# Create grid for plotting
lon_grid = np.linspace(-10, 10, 50)
lat_grid = np.linspace(-10, 10, 50)
lon, lat = np.meshgrid(lon_grid, lat_grid)
grid = np.column_stack([
    np.zeros_like(lon.ravel()),  # X1 set to 0 to isolate spatial effect
    lon.ravel(),
    lat.ravel()
])

# Predict spatial effect
spatial_preds = gam.predict(grid).reshape(lon.shape)

# Plot
plt.figure(figsize=(8, 6))
plt.contourf(lon, lat, spatial_preds, levels=20, cmap='viridis')
plt.colorbar(label='Predicted y (spatial effect)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Tensor Product Smooth over Space')
plt.show()


In [ ]:
X = df[["X1","lat","lon"]].values.reshape(-1,3)
y = df["y_d"].values.reshape(-1,1)
X


In [ ]:
# Sort and extract variables
gdf = gdf.sort_values(["time", "id"]).reset_index(drop=True)

# Encode spatial unit ids as integers 0..N-1
gdf["unit_id"] = gdf["id"].astype("category").cat.codes
N = gdf["unit_id"].nunique()
T = gdf["time"].nunique()

y = gdf["y_d"].values
X1 = gdf["X1"].values
Wy = gdf["w_d"].values
unit_idx = gdf["unit_id"].values


In [ ]:
with pm.Model() as model:
    # Hyperpriors
    sigma = pm.HalfNormal("sigma", 2.0)
    tau_rho = pm.HalfNormal("tau_rho", 1.0)
    tau_mu = pm.HalfNormal("tau_mu", 1.0)

    # Priors
    beta = pm.Normal("beta", mu=0, sigma=5)
    rho_i = pm.Normal("rho", mu=0, sigma=tau_rho, shape=N)     # one rho per unit
    mu_i = pm.Normal("mu", mu=0, sigma=tau_mu, shape=N)         # one intercept per unit

    # Create shared inputs
    X_data = pm.Data("X1", X1)
    Wy_data = pm.Data("Wy", Wy)
    unit_idx_data = pm.Data("unit_idx", unit_idx)

    # Compute mu_y
    mu_y = rho_i[unit_idx_data] * Wy_data + beta * X_data + mu_i[unit_idx_data]

    # Likelihood
    y_obs = pm.Normal("y_obs", mu=mu_y, sigma=sigma, observed=y)

    trace = pm.sample(1000, tune=1000, target_accept=0.9, return_inferencedata=True)


In [ ]:
# az.plot_trace(trace, var_names=["rho", "beta", "sigma"])
# az.summary(trace, var_names=["rho", "beta", "sigma"])


In [ ]:
rho_true = .8
summary = az.summary(trace, var_names=["rho"], hdi_prob=0.94)
within_hdi = (rho_true >= summary["hdi_3%"]) & (rho_true <= summary["hdi_97%"])

# Report results
all_contain = within_hdi.all()
num_pass = within_hdi.sum()
num_total = len(within_hdi)

print(f"True rho = {rho_true}")
print(f"{num_pass}/{num_total} HDIs contain true rho.")

# Optionally, list which units failed
if not all_contain:
    failed_units = np.where(~within_hdi)[0]
    print(f"Units failing HDI test: {failed_units}")